# １．　クローラをもっと進化させるには
保守、運用を考慮した設計をしていく必要がある

# ２．　print関数でログを出力する

In [12]:
import requests
import lxml.html
from icecream import ic #print関数の代わりに使用

#HTMLソースを得る
r=requests.get('https://www.shoeisha.co.jp/book/detail/9784798146072')
html=r.text

#HTMLをHTML Elementオブジェクトにする
root=lxml.html.fromstring(html)

#Xpathを指定して該当する要素のリストを得る
titleH1=root.xpath('//*[@id="cx_contents_block"]/div[1]/section/h1')
#リストの１番目の要素のテキストを表示する
ic(titleH1[0].text)

# print(titleH1[0].tag)
# print(titleH1[0].attrib)

#CSSセレクターで該当する要素のリストを得る
qaA=root.cssselect('#qa > p:nth-child(2) > a')

#取得した要素のhref要素を得る
for aTag in qaA:
    ic(aTag.attrib['href'])

ic| titleH1[0].text: 'iPhone＆Androidアプリ内課金プログラミング完全ガイド 第2版
                                    '
ic| aTag.attrib['href']: 'https://www.shoeisha.co.jp/book/qa/form/9784798146072'


## ３．　loggingモジールでログを取得して管理する

In [17]:
from logging import (
    getLogger,
    Formatter,
    FileHandler,
    StreamHandler,
    DEBUG,
    ERROR,
    )

import requests

#ロガー：　__name__には実行モジュール名logging_sampleが入る
logger=getLogger(__name__)

#出力フォーマット
default_format='[%(levelname)s] %(asctime)s %(name)s %(filename)s:%(lineno)d %(message)s'
default_formatter=Formatter(default_format)
funcname_formatter=Formatter(default_format+'%(funcName)s')

#ログ用ハンドラー：　コンソール出力用
log_stream_handler=StreamHandler()
log_stream_handler.setFormatter(default_formatter)
log_stream_handler.setLevel(DEBUG)

#ログ用ハンドラー：　ファイル出力用
log_file_handler=FileHandler(filename='crawler.log')
log_file_handler.setFormatter(funcname_formatter)
log_file_handler.setLevel(ERROR)

#ロガーにハンドラーとレベルをセット
logger.setLevel(DEBUG)
logger.addHandler(log_stream_handler)
logger.addHandler(log_file_handler)

def logging_example():
    logger.info('クロールを開始します') #INFOレベルでメッセージを出力する
    logger.warning('外部サイトのリンクのためクロールしません')  #WARNINGレベルでメッセージを出力する
    logger.error('ページが見つかりませんでした')    #ERRORレベルでメッセージを出力する

    try:
        r=requests.get('#invalid_url',timeout=1)
    except requests.exceptions.RequestException as e:
        logger.exception('リクエスト中に例外が起きました: %r',e)    #例外のスタックトレースを出力する


if __name__=='__main__':
    logging_example()

[INFO] 2021-02-12 18:50:47,716 __main__ <ipython-input-17-dac7d459b6aa>:36 クロールを開始します
[INFO] 2021-02-12 18:50:47,716 __main__ <ipython-input-17-dac7d459b6aa>:36 クロールを開始します
[INFO] 2021-02-12 18:50:47,716 __main__ <ipython-input-17-dac7d459b6aa>:36 クロールを開始します
[WARNING] 2021-02-12 18:50:47,729 __main__ <ipython-input-17-dac7d459b6aa>:37 外部サイトのリンクのためクロールしません
[WARNING] 2021-02-12 18:50:47,729 __main__ <ipython-input-17-dac7d459b6aa>:37 外部サイトのリンクのためクロールしません
[WARNING] 2021-02-12 18:50:47,729 __main__ <ipython-input-17-dac7d459b6aa>:37 外部サイトのリンクのためクロールしません
[ERROR] 2021-02-12 18:50:47,733 __main__ <ipython-input-17-dac7d459b6aa>:38 ページが見つかりませんでした
[ERROR] 2021-02-12 18:50:47,733 __main__ <ipython-input-17-dac7d459b6aa>:38 ページが見つかりませんでした
[ERROR] 2021-02-12 18:50:47,733 __main__ <ipython-input-17-dac7d459b6aa>:38 ページが見つかりませんでした
[ERROR] 2021-02-12 18:50:47,740 __main__ <ipython-input-17-dac7d459b6aa>:43 リクエスト中に例外が起きました: MissingSchema("Invalid URL '#invalid_url': No schema supplied. Perhaps you mean

In [14]:
!python logging_sample.py

/System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: can't open file 'logging_sample.py': [Errno 2] No such file or directory
